In [1]:
import elasticsearch
import os
import ssl

In [2]:
es = elasticsearch.Elasticsearch(["http://localhost:9200"]) 


In [3]:
# Autentificare cu parola: de verificat!!

es = elasticsearch.Elasticsearch(hosts=os.environ.get("ES_HOST", "https://localhost:9200"),
basic_auth=( 'elastic', 'MemdLYXUN5-PpG5tVZ9E'

),
ssl_version=ssl.TLSVersion.TLSv1_2,
verify_certs=False)

index_name = "my_index"

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/elasticsearch/_sync/client/__init__.py:311: SecurityWarning: Connecting to 'https://localhost:9200' using TLS with verify_certs=False is insecure
  _transport = transport_class(


In [4]:
es

<Elasticsearch(['https://localhost:9200'])>

In [5]:
#
es.indices.exists(index='locuri')

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


HeadApiResponse(False)

In [6]:
es.indices.create(index="python_index")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'python_index'})

In [7]:
es.cluster.health(wait_for_status="yellow")

/Library/Frameworks/Python.framework/Versions/3.13/lib/python3.13/site-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'localhost'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


ConnectionTimeout: Connection timed out

In [ ]:
index_name = "my_index"
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
es.indices.create(index=index_name)
es.cluster.health(wait_for_status="yellow")

In [ ]:
es.indices.put_mapping(index=index_name,
    body={
        "properties":{
            "uuid":{
                "type":"keyword"
            },
            "title":{
                "type":"text",
                "term_vector":"with_positions_offsets"
            },
            "parsedtext":{
                "type":"text",
                "term_vector":"with_positions_offsets"
            },
            "nested":{
                "type":"nested",
                "properties":{
                    "num":{
                    "type":"integer"
                    },
                    "name":{
                    "type":"keyword"
                    },
                    "value":{
                    "type":"keyword"
                    }
                }
            },
            "date":{
                "type":"date"
            },
            "position":{
                "type":"integer"
            },
            "name":{
                "type":"text",
                "term_vector":"with_positions_offsets"
            }
        }
        }
)

In [ ]:
mappings = es.indices.get_mapping(index=index_name)
mappings

## Managing documents

In [ ]:
es.index(index=index_name, id="2",
    body={"name": "data2", "value": "value2", "join_field": {"name": "metadata", "parent": "2"}},
routing=2)

In [ ]:
es.delete(index=index_name, id="2")

In [ ]:
results = es.search(index=index_name)
results

In [ ]:
from elasticsearch.helpers import bulk
from datetime import datetime
bulk(es, [{
   "_index":"index_name",
   "_id":"1",
   "source":{
      "name":"Joe Tester",
      "parsedtext":"Joe Testere nice guy",
      "uuid":"11111",
      "position":1,
      "date":datetime(2018, 12, 8)
   }
},
{
   "_index":"index_name",
   "_id":"2",
   "source":{
      "name":"Bill Baloney",
      "parsedtext":"Bill Testere nice guy",
      "uuid":"22222",
      "position":2,
      "date":datetime(2018, 12, 8)
   }
}])

In [ ]:
es.update(index=index_name,  id="2", body={"script": 'ctx._source.position += 1'})